<a href="https://colab.research.google.com/github/Tariquzzaman-faisal/hatespeech/blob/main/official_embedding_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pandas

In [3]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/bengali_hate_v2.0.csv")

In [4]:
!pip install setuptools
!pip install git+https://github.com/csebuetnlp/normalizer
!pip install pandas transformers torch

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-l5zqp2m5
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-l5zqp2m5
  Resolved https://github.com/csebuetnlp/normalizer to commit d80c3c484e1b80268f2b2dfaf7557fe65e34f321
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6860 sha256=dae50c33c5053e54bb99578ca43369eea29304025b27fc6b5bec8c1171d57ee7
  Stored in directory: /tmp/pip-ephem-wheel-cache-8tkbb_xx/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186456 sha256=ee322e6ee3313fc9c8139fe33910017752fdc647

In [5]:
from transformers import AutoModelForPreTraining, AutoTokenizer
from normalizer import normalize
import torch

In [6]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('GPU is available!')
else:
    device = torch.device('cpu')
    print('No GPU found, using CPU.')

GPU is available!


In [7]:
MODEL_NAME = "csebuetnlp/banglabert"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# Load the pre-trained model and move it to the GPU
model = AutoModelForPreTraining.from_pretrained(MODEL_NAME)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

ElectraForPreTraining(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((7

In [8]:
df['text'][1]

'এই সরকার কে যারা নির্লজ্জের মত সাপোর্ট দিয়েছে বছরের পর বছর, তাদের আরো এমন রাস্তায় রাস্তায় কাঁদতে হবে '

In [15]:
import torch
import numpy as np

# Check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the GPU
model.to(device)

embeddings_list = []  # Temporary list to store embeddings
targets_list = []     # Temporary list to store targets

for index, row in df.iterrows():
    text = row['text']
    target = row['target']
    text = normalize(text) # this normalization step is required before tokenizing the text
    tokens = tokenizer.tokenize(text)
    encoded_inputs = tokenizer.encode(text, return_tensors="pt").to(device)  # Move input tensor to the GPU
    embedding = model(encoded_inputs).logits
    embeddings_list.append(embedding.cpu().detach().numpy())  # Move the embedding back to the CPU and convert to a NumPy array
    targets_list.append(target)
    if index % 100 == 0:
        print(index)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600


In [20]:
# Convert embeddings and targets to NumPy arrays after the loop
print(embeddings_list[:2])
print(targets_list[:2])

[array([[-13.003     ,  -0.73461753,  -0.5727064 ,  -3.808109  ,
         -3.8040235 ,  -2.9668963 ,  -3.6879823 ,  -5.5295258 ,
         -4.7620587 ,  -5.0254316 ,  -3.465695  ,  -6.2412205 ,
         -2.9543893 ,  -4.3848205 ,  -2.039203  ,  -4.2219925 ,
         -3.5316584 ,  -5.695008  ,  -3.590968  ,  -1.4514673 ,
        -13.002878  ]], dtype=float32), array([[-12.778679  ,  -2.689206  ,  -3.3512566 ,  -5.0513325 ,
         -5.888815  ,  -3.6059506 ,  -5.3089685 ,  -3.2414424 ,
         -3.5210378 ,  -2.3684523 ,  -7.166422  ,  -7.1795263 ,
         -6.812591  ,  -3.7075176 ,  -3.2538033 ,   0.32642305,
         -0.16260503,  -3.0317822 ,  -2.2290273 ,  -1.7612277 ,
         -4.2049713 , -12.778489  ]], dtype=float32)]
[0, 1]


In [21]:
import numpy as np

def normalize_embeddings(embeddings_list):
    normalized_embeddings = []
    for emb in embeddings_list:
        norm = np.linalg.norm(emb, ord=2)  # Calculate L2 norm
        normalized_emb = emb / norm if norm != 0 else emb
        normalized_embeddings.append(normalized_emb)
    return normalized_embeddings

# Assuming bert_embeddings is the list of BERT embeddings for your dataset
normalized_bert_embeddings = normalize_embeddings(embeddings_list)


In [32]:
print(type(normalized_bert_embeddings[2]))
print(type(normalized_bert_embeddings))
print(type(targets_list[2]))
print(type(targets_list))
print(normalized_bert_embeddings[0].shape)

<class 'numpy.ndarray'>
<class 'list'>
<class 'int'>
<class 'list'>
(1, 21)


In [41]:
directory = "/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/dataset"
np.savez(f'{directory}/embeddings.npz', *normalized_bert_embeddings)
np.save(f'{directory}/targets.npy',targets_list)

In [42]:
directory = "/content/drive/MyDrive/Research/Shared Task/Violence Inciting Text Detection (VITD) Bangla/dataset"

# Load the embeddings data
loaded_data = np.load(f'{directory}/embeddings.npz')

# Get the keys of the loaded_data, which are in the format 'arr_0', 'arr_1', ... 'arr_n'
embedding_keys = loaded_data.files

# The number of embeddings is equal to the length of the embedding_keys
num_embeddings = len(embedding_keys)

# Load each embedding into a list
embeddings_list_new = [loaded_data[key] for key in embedding_keys]

# Load targets
targets_list_new = np.load(f'{directory}/targets.npy')


In [43]:
print(embeddings_list_new[:5])

[array([[-0.5181694 , -0.0292745 , -0.02282234, -0.1517531 , -0.15159029,
        -0.11823078, -0.14696606, -0.22035153, -0.18976799, -0.2002634 ,
        -0.1381079 , -0.24871255, -0.11773238, -0.17473504, -0.08126221,
        -0.16824634, -0.14073655, -0.226946  , -0.14310002, -0.05784095,
        -0.5181645 ]], dtype=float32), array([[-0.48872364, -0.10284932, -0.12816961, -0.19318941, -0.22521913,
        -0.13791044, -0.20304278, -0.12396974, -0.13466294, -0.09058203,
        -0.27408153, -0.2745827 , -0.2605492 , -0.1417949 , -0.12444249,
         0.01248413, -0.00621887, -0.11595123, -0.08524968, -0.06735858,
        -0.16082014, -0.48871636]], dtype=float32), array([[-0.55080575, -0.30112183, -0.14407268, -0.14417697, -0.170719  ,
        -0.0194599 , -0.13347523, -0.16400942, -0.10932852, -0.1131859 ,
        -0.09428434, -0.10111327, -0.11169581, -0.28146273, -0.16224246,
        -0.14642157, -0.05860957, -0.5508039 ]], dtype=float32), array([[-0.47177735, -0.11900678, -0.159

In [44]:
print(targets_list_new)

[0 1 3 ... 0 0 1]
